In [26]:
import time
from datetime import datetime, timedelta
import requests
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
logging = logging.getLogger(__name__)


yesterday_noon = datetime.now().replace(hour=12, minute=0, second=0, microsecond=0) - timedelta(days=1)
unix_time = int(yesterday_noon.timestamp())
print(unix_time)

1755619200


In [15]:
from data.fetch_data import bulk_fetch_matches

In [16]:
pred_matches = bulk_fetch_matches(
    max_days_fetch = 30,
    min_days = 7,
    max_days = 5
)


DEBUG:root:
Batch 0: fetching day from 5 to 6 days ago
DEBUG:root:Fetching match data..
DEBUG:root:Time range: 5 days ago to 6 days ago
DEBUG:root:Unix timestamps: 1755315797 to 1755229397
INFO:root:Making request to: https://api.deadlock-api.com/v1/matches/metadata?include_player_info=true&min_unix_timestamp=1755229397&max_unix_timestamp=1755315797&min_average_badge=100&limit=500
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.deadlock-api.com:443
DEBUG:root:Fetching match data..
DEBUG:root:Time range: 5 days ago to 6 days ago
DEBUG:root:Unix timestamps: 1755315797 to 1755229397
INFO:root:Making request to: https://api.deadlock-api.com/v1/matches/metadata?include_player_info=true&min_unix_timestamp=1755229397&max_unix_timestamp=1755315797&min_average_badge=100&limit=500
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.deadlock-api.com:443


DEBUG: Fetching matches for day 1 from 5 to 6 days ago


DEBUG:urllib3.connectionpool:https://api.deadlock-api.com:443 "GET /v1/matches/metadata?include_player_info=true&min_unix_timestamp=1755229397&max_unix_timestamp=1755315797&min_average_badge=100&limit=500 HTTP/1.1" 200 None
DEBUG:root:
Batch 1: fetching day from 6 to 7 days ago
DEBUG:root:
Batch 1: fetching day from 6 to 7 days ago
DEBUG:root:Fetching match data..
DEBUG:root:Time range: 6 days ago to 7 days ago
DEBUG:root:Unix timestamps: 1755229397 to 1755142997
INFO:root:Making request to: https://api.deadlock-api.com/v1/matches/metadata?include_player_info=true&min_unix_timestamp=1755142997&max_unix_timestamp=1755229397&min_average_badge=100&limit=500
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.deadlock-api.com:443
DEBUG:root:Fetching match data..
DEBUG:root:Time range: 6 days ago to 7 days ago
DEBUG:root:Unix timestamps: 1755229397 to 1755142997
INFO:root:Making request to: https://api.deadlock-api.com/v1/matches/metadata?include_player_info=true&min_unix_ti

DEBUG: Fetching matches for day 2 from 6 to 7 days ago


DEBUG:urllib3.connectionpool:https://api.deadlock-api.com:443 "GET /v1/matches/metadata?include_player_info=true&min_unix_timestamp=1755142997&max_unix_timestamp=1755229397&min_average_badge=100&limit=500 HTTP/1.1" 200 None


Reached configured minimum day boundary (7 days ago)


In [17]:
for idx, d in enumerate(pred_matches):
    print(f"Day {idx}:")
    for match in d:
        print(f"  Match ID: {match['match_id']}, Players: {[player['account_id'] for player in match['players']]}")


Day 0:
  Match ID: 38819287, Players: [123795813, 278919272, 877571505, 1864276769, 1285605078, 859118081, 1595498032, 1885262860, 108710262, 1862446802, 108815945, 1832319597]
  Match ID: 38819410, Players: [150558972, 55297093, 181456277, 904121169, 918074721, 141706498, 394096290, 300474764, 328475680, 108264063, 1056512979, 957622850]
  Match ID: 38819440, Players: [85618996, 195257679, 1924675708, 37635280, 352231929, 1872447203, 1926422616, 347624260, 162425986, 1924808882, 136497875, 250549491]
  Match ID: 38819477, Players: [1715337879, 384653741, 1125297246, 837058017, 308362270, 1846909634, 1167486585, 1896932924, 381164128, 329035753, 339976259, 349508451]
  Match ID: 38819663, Players: [895588232, 1862853193, 39366096, 404130420, 1141067246, 1846180210, 111701409, 1857321245, 1176357514, 173357955, 86240532, 54768669]
  Match ID: 38819769, Players: [1885262860, 85898225, 87839692, 278919272, 1925530166, 1108205529, 877571505, 1529829716, 96398669, 1113450763, 1588365521, 10

In [18]:
unique_players = set()
for d in pred_matches:
    for match in d:
        unique_players.update(player['account_id'] for player in match['players'])
print(f"Unique players identified: {len(unique_players)}")

Unique players identified: 1905


In [19]:
old_players = pd.read_csv("v2_data\\8.12.data\\v2\\phm_stats.8.12_v2.csv")

In [20]:
# compare account_id in set unique_players with df old_players, counting how many are missing between the two
old = set(old_players['account_id'])
# unique_players is already a set from previous cell

missing_in_old = unique_players - old
missing_in_new = old - unique_players

print(f"Players in unique_players but not in old_players: {len(missing_in_old)}")
print(f"Players in old_players but not in unique_players: {len(missing_in_new)}")
print(f"Missing in old_players: {missing_in_old}")
print(f"Missing in unique_players: {missing_in_new}")

Players in unique_players but not in old_players: 876
Players in old_players but not in unique_players: 1404
Missing in old_players: {117151746, 406470661, 53194758, 204832775, 1742262279, 1192380425, 53612554, 49653771, 34566154, 448618509, 66646029, 1207703560, 333879309, 914798612, 188516373, 388390934, 293806106, 122953761, 321562658, 277749800, 898877481, 1894436904, 133685291, 87037994, 64092204, 173445172, 323967030, 1007573047, 311672893, 1804691520, 53835841, 255932485, 43104332, 186157133, 1869183052, 190664786, 298872917, 1270505558, 430424155, 936546405, 1042911338, 324350063, 47075444, 1253533815, 1924675708, 1686411392, 255389825, 248881284, 26452101, 1046962308, 91172996, 450865288, 48451716, 308723850, 1564756106, 1741691026, 212811923, 1713834132, 1924458645, 857106582, 1874819221, 271679640, 241911976, 1131264169, 181577898, 1277141168, 907231409, 1924808882, 212973745, 430166196, 335925433, 173383867, 1903235260, 125755581, 443128000, 318298306, 1223276739, 170884115

In [ ]:
old_players

In [21]:
from data.process_data import separate_match_players

matches, players = separate_match_players(pred_matches)

INFO:root:Normalizing bulk match data
INFO:root:Processing day #0 with 199 matches
INFO:root:Processing day #1 with 258 matches
INFO:root:Processing day #0 with 199 matches
INFO:root:Processing day #1 with 258 matches


In [22]:
# Add win column
if not matches.empty and not players.empty:
    players = players.merge(
        matches[['match_id', 'winning_team']],
        on='match_id',
        how='left'
    )
    players['win'] = players.apply(
        lambda row: 'Y' if row['team'] == row['winning_team'] else 'N',
        axis=1
    )

In [23]:
from datetime import datetime
now = datetime.now()
date = now.date()
hour = now.time()

now = datetime.now()
matches.to_csv(f"v2_data\\8.12.data\\pred_datamatches_{date}.csv", index=False)
players.to_csv(f"v2_data\\8.12.data\\pred_dataplayers_{date}.csv", index=False)

In [ ]:
# fetch data from CSV
### Checkpoint ###
players = pd.read_csv("v2_data\\8.12.data\\pred_data\\pred_dataplayers_2025-08-18.csv")
#

In [62]:
def get_unix_time(days_ago=0):   
    """
    Convert days_ago to a unix timestamp
    
    Parameters:
    days_ago (int): Number of days in the past
    
    Returns:
    int: Unix timestamp for the date that was 'days_ago' days ago
    """
    current_time = int(time.time())
    seconds_ago = int(timedelta(days=days_ago).total_seconds())
    return current_time - seconds_ago

def fetch_player_hero_stats(account_id, start_date=None, end_date=None) -> dict:
    """Fetches hero stats for a specific player from the Deadlock API.
    Generally used in conjunction with run_player_batches and 
    process_player_stats_parallel

    - account_id: Player's account ID to fetch stats for (can be string or numeric)
    
    Returns:
    - JSON response containing player's hero stats or error dict
    """
    
    base = "https://api.deadlock-api.com/v1/players/hero-stats?account_ids="
    
    if start_date is not None:
        older_time = str(f"?min_unix_timestamp={get_unix_time(start_date)}")
    else:
        older_time = ""

    if end_date is not None:
        newer_time = str(f"&max_unix_timestamp={get_unix_time(end_date)}")
    else:
        newer_time = ""

    # Force acocunt_id to str
    account_ids = ",".join(str(x) for x in account_id['account_id'].unique()[:1000])

    path = f"{base}{account_ids}{older_time}{newer_time}"
    
    try:
        #logging.debug(f"Fetching stats for player {account_id_str}")
        response = requests.get(path)
        if response.status_code != 200:
            logging.error(f"API request failed for player {account_ids} with status code {response.status_code}")
            logging.error(f"Response: {response.text}")
            return {"error": f"API request failed with status code {response.status_code}"}
        return response.json()
    except Exception as e:
        logging.error(f"Exception fetching hero stats for player {account_ids}: {e}")
        return {"error": str(e)}


def process_player_stats_parallel(player_ids,start_date, end_date, max_workers=50, timeout=30):
    """
    Fetches and processes hero stats for multiple players in parallel
    
    Parameters:
    - player_ids: List of player account IDs to fetch stats for
    - max_workers: Maximum number of concurrent requests
    - timeout: Timeout in seconds for each request
    
    Returns:
    - Tuple of (player_stats_df, player_hero_stats_df)
    """

    import concurrent.futures
    import time
    from tqdm.notebook import tqdm

    player_stats = []
    player_hero_stats = []
    error_count = 0
    
    # Define stats we want to aggregate
    stats_columns = [
        'matches_played', 'wins', 'kills', 'deaths', 'assists',
        'damage_per_min', 'time_played'
    ]
    
    # Convert all player IDs to strings
    player_ids_str = [str(pid) for pid in player_ids]
    
    logging.info(f"Processing {len(player_ids_str)} players with {max_workers} workers")
    
    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and store futures
        future_to_id = {
            executor.submit(fetch_player_hero_stats, player_id, start_date, end_date): player_id
            for player_id in player_ids_str
        }
        
        # Process results as they complete
        for future in tqdm(concurrent.futures.as_completed(future_to_id), total=len(player_ids_str)):
            player_id = future_to_id[future]
            
            try:
                result = future.result(timeout=timeout)
                
                # Skip if error
                if "error" in result:
                    error_count += 1
                    continue
                
                # Skip if empty response
                if not result:
                    logging.warning(f"Empty result for player {player_id}")
                    continue
                
                # Calculate aggregate stats per player
                player_matches_played = sum(hero_stat.get('matches_played', 0) for hero_stat in result)
                if player_matches_played == 0:
                    # Skip players with no matches
                    logging.warning(f"Player {player_id} has no matches")
                    continue
                    
                total_kills = sum(hero_stat.get('kills', 0) for hero_stat in result)
                total_deaths = sum(hero_stat.get('deaths', 0) for hero_stat in result)
                total_wins = sum(hero_stat.get('wins', 0) for hero_stat in result)
                total_assists = sum(hero_stat.get('assists', 0) for hero_stat in result)
                total_time_played = sum(hero_stat.get('time_played', 0) for hero_stat in result)
                
                # Calculate averages
                avg_kd = total_kills / max(total_deaths, 1)  # Avoid division by zero
                win_rate = total_wins / player_matches_played if player_matches_played > 0 else 0
                
                # Add to player stats dataframe
                player_stats.append({
                    'account_id': player_id,
                    'matches_played': player_matches_played,
                    'total_kills': total_kills,
                    'total_deaths': total_deaths,
                    'total_assists': total_assists,
                    'avg_kd': avg_kd,
                    'win_rate': win_rate,
                    'total_time_played': total_time_played
                })
                
                # Add hero-specific stats to player_hero_stats dataframe
                for hero_stat in result:
                    hero_id = hero_stat.get('hero_id')
                    if hero_id is not None:
                        # Extract valid stats, handling potential missing keys
                        valid_stats = {
                            col: hero_stat.get(col, 0) for col in stats_columns
                        }
                        
                        # Add entry to hero stats
                        player_hero_stats.append({
                            'account_id': player_id,
                            'hero_id': hero_id,
                            **valid_stats
                        })
                
            except Exception as e:
                logging.error(f"Error processing player {player_id}: {e}")
                error_count += 1

    logging.info(f"Completed with {error_count} errors out of {len(player_ids_str)} players")

    # Convert to DataFrames
    df_player_stats = pd.DataFrame(player_stats) if player_stats else pd.DataFrame()
    df_player_hero_stats = pd.DataFrame(player_hero_stats) if player_hero_stats else pd.DataFrame()
    
    return df_player_stats, df_player_hero_stats

def t_run_player_batches(player_ids, start_date, end_date, batch_size=1000, max_workers_per_batch=15, timeout=30):
    """
    Process all players in batches
    
    Parameters:
    - player_ids: List of player IDs to process
    - batch_size: Number of players to process in each batch
    - max_workers_per_batch: Number of concurrent workers per batch
    - timeout: Request timeout in seconds
    
    Returns:
    - Tuple of (player_stats_df, player_hero_stats_df)
    """

    num_batches = (len(player_ids) + batch_size - 1) // batch_size  # Ceiling division

    batched_player_stats = []
    batched_player_hero_stats = []
    successful_players = 0
    failed_players = 0

    logging.info(f"Processing {len(player_ids)} players in {num_batches} batches of {batch_size}")

    total_start_time = time.time()

    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, len(player_ids))
        batch_players = player_ids[start_idx:end_idx]

        logging.info(f"Starting batch {batch_num+1}/{num_batches} with {len(batch_players)} players")

        batch_start_time = time.time()
        
        # Process batch
        batch_player_stats, batch_player_hero_stats = fetch_player_hero_stats(
            batch_players, 
            start_date,
            end_date,
        )
        
        batch_end_time = time.time()
        batch_duration = batch_end_time - batch_start_time

        batch_successful = len(batch_player_stats) if not batch_player_stats.empty else 0
        batch_failed = len(batch_players) - batch_successful
        
        successful_players += batch_successful
        failed_players += batch_failed

        # Summary of batch success
        success_rate = (batch_successful / len(batch_players)) * 100 if len(batch_players) > 0 else 0
        logging.info(f"Batch {batch_num+1}/{num_batches}: {batch_successful}/{len(batch_players)} players processed ({success_rate:.1f}%) in {batch_duration:.2f} seconds")
        
        if not batch_player_stats.empty:
            hero_stats_per_player = len(batch_player_hero_stats) / batch_successful if batch_successful > 0 else 0
            #logging.info(f"  Avg hero entries per player: {hero_stats_per_player:.1f}")
            
            batched_player_stats.append(batch_player_stats)
            batched_player_hero_stats.append(batch_player_hero_stats)

    # Combine all batches
    if batched_player_stats:
        combined_player_stats = pd.concat(batched_player_stats, ignore_index=True)
        combined_player_hero_stats = pd.concat(batched_player_hero_stats, ignore_index=True)
        
        total_end_time = time.time()
        total_duration = total_end_time - total_start_time
        
        # Overall summary
        total_success_rate = (successful_players / len(player_ids)) * 100 if len(player_ids) > 0 else 0
        logging.info(f"\n=== Processing Summary ===")
        logging.info(f"Total time: {total_duration:.2f} seconds")
        logging.info(f"Players processed: {successful_players}/{len(player_ids)} ({total_success_rate:.1f}%)")
        logging.info(f"Hero entries: {len(combined_player_hero_stats)} (avg {len(combined_player_hero_stats)/successful_players:.1f} per player)")
        
        if successful_players > 0:
            logging.info(f"Average K:D ratio: {combined_player_stats['avg_kd'].mean():.2f}")
            logging.info(f"Average win rate: {combined_player_stats['win_rate'].mean():.2f}")

        return combined_player_stats, combined_player_hero_stats
    
    else:
        logging.error("No player data was successfully processed")
        return pd.DataFrame(), pd.DataFrame()
    
    

In [64]:
#testing function:
run_test = players['account_id'].unique()[:10]  # Use a small sample for testing


test_player_stats, test_player_hero_stats = t_run_player_batches(
    players['account_id'].nunique(),
    start_date = 1,
    end_date = None,
)

# Save test data
test_player_stats.to_csv("v2_data/test_player_stats.csv", index=False)
test_player_hero_stats.to_csv("v2_data/test_player_hero_stats.csv", index=False)


INFO:__main__:Processing 5484 players in 6 batches of 1000
INFO:__main__:Starting batch 1/6 with 1000 players


KeyError: 'account_id'

In [ ]:
import logging
import pandas as pd
import requests

# Configure root logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

urllib3_logger = logging.getLogger('urllib3')
urllib3_logger.setLevel(logging.WARNING)

player_stats, player_hero_stats = t_run_player_batches(
    players['account_id'].unique(),
    start_date = 1,
    end_date = None,
    batch_size = 15,
)

INFO:root:Processing 1983 players in 133 batches of 15
INFO:root:Starting batch 1/133 with 15 players
INFO:root:Processing 15 players with 15 workers
INFO:root:Starting batch 1/133 with 15 players
INFO:root:Processing 15 players with 15 workers


  0%|          | 0/15 [00:00<?, ?it/s]

ERROR:root:API request failed for player 1356683901 with status code 404
ERROR:root:Response: 
ERROR:root:Response: 
ERROR:root:API request failed for player 1032997490 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 256512296 with status code 404
ERROR:root:API request failed for player 1032997490 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 256512296 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1475821079 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1848503388 with status code 404
ERROR:root:API request failed for player 1854731997 with status code 404
ERROR:root:API request failed for player 1610485462 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1475821079 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1848503388 with status code 404
ERROR:root:API request 

  0%|          | 0/15 [00:00<?, ?it/s]

ERROR:root:API request failed for player 1247756097 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1513639972 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1513639972 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 66646029 with status code 404
ERROR:root:Response: 
ERROR:root:Response: 
ERROR:root:API request failed for player 66646029 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 158057048 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 158057048 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 186157133 with status code 404
ERROR:root:API request failed for player 1742262279 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 34262576 with status code 404
ERROR:root:API request failed for player 1186801441 with status code 404
ERROR:ro

...

  0%|          | 0/15 [00:00<?, ?it/s]

ERROR:root:API request failed for player 1682265751 with status code 404
ERROR:root:Response: 
ERROR:root:Response: 
ERROR:root:API request failed for player 112724001 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1274954495 with status code 404
ERROR:root:API request failed for player 112724001 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1274954495 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 289323388 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 919161010 with status code 404
ERROR:root:API request failed for player 207697416 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 289323388 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 919161010 with status code 404
ERROR:root:API request failed for player 207697416 with status code 404
ERROR:root:Response: 
ERROR:r

  0%|          | 0/15 [00:00<?, ?it/s]

ERROR:root:API request failed for player 1661193915 with status code 404
ERROR:root:Response: 
ERROR:root:Response: 
ERROR:root:API request failed for player 52545810 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 52545810 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1035409245 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 423990470 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1476238442 with status code 404
ERROR:root:API request failed for player 1035409245 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 423990470 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1476238442 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1840078257 with status code 404
ERROR:root:Response: 
ERROR:root:API request failed for player 1478734915 with

KeyboardInterrupt: 

In [ ]:
# Save data
player_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_stats.csv", index=False)
player_hero_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_hero_stats.csv", index=False)


In [ ]:
def retry_failed_players(start_date, end_date, player_hero_stats, original_player_set, completed_player_stats):
    """
    Identifies player IDs that failed to process and retries them
    
    This function:
    1. Finds all player IDs in the original dataset that aren't in the processed results
    2. Retries processing these failed players in smaller batches
    3. Combines the retry results with the original results
    4. Saves the updated complete dataset
    """
    print("Starting retry process for failed player IDs...")
    
    def load_df(input_val):
        if isinstance(input_val, pd.DataFrame):
            return input_val.copy()
        elif isinstance(input_val, str) and input_val.lower().endswith('.csv'):
            try:
                return pd.read_csv(input_val)
            except Exception as e:
                print(f"Error reading CSV file {input_val}: {e}")
                return None
        else:
            print(f"Incorrect variable type for input: {type(input_val)}. Must be DataFrame or .csv path.")
            return None

    try:
        # Load the original unique player IDs from the match data
        original_players_df = load_df(original_player_set)
        if original_players_df is None:
            print("Failed to load original player set. Exiting.")
            return
        all_original_players = set(original_players_df['account_id'].astype(str).unique())
        print(f"Total unique players in original data: {len(all_original_players)}")
    
        # load the processed player stats
        processed_stats = load_df(completed_player_stats)
        if processed_stats is None:
            print("Failed to load processed player stats. Exiting.")
            return
        processed_player_ids = set(processed_stats['account_id'].astype(str))
        print(f"Successfully processed players: {len(processed_player_ids)}")
        
        # Find failed players (in original set but not in processed set)
        failed_player_ids = list(all_original_players - processed_player_ids)
        print(f"Found {len(failed_player_ids)} failed player IDs to retry")
        
        if not failed_player_ids:
            print("No failed players to retry. All players were processed successfully.")
            return
            
        # Process the failed players with smaller batch size and more timeout
        print("\n--- Retrying failed players ---")
        retry_stats, retry_hero_stats = t_run_player_batches(
            failed_player_ids,
            start_date = start_date,
            end_date = end_date,
            batch_size=50,            
            max_workers_per_batch=10,  
            timeout=45                
        )
        
        # Check if retry was successful
        if retry_stats.empty:
            print("Retry process didn't yield any successful results")
            return
            
        # Combine the retry results with the original results
        print("\nCombining retry results with original results...")
        
        # Load original hero stats
        processed_hero_stats = player_hero_stats.copy()
        
        # Combine data
        combined_stats = pd.concat([processed_stats, retry_stats], ignore_index=True)
        combined_hero_stats = pd.concat([processed_hero_stats, retry_hero_stats], ignore_index=True)
        
        # Save the updated complete dataset
        #### Hardcoded Save ####
        combined_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_stats_with_retries.csv", index=False)
        combined_hero_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_hero_stats_with_retries.csv", index=False)
        
        # Calculate success rate
        total_success_count = len(combined_stats)
        success_rate = total_success_count / len(all_original_players) * 100
        
        print("\n--- Retry Summary ---")
        print(f"Original successful players: {len(processed_player_ids)}")
        print(f"Additional players from retry: {len(retry_stats)}")
        print(f"Total successful players: {total_success_count}/{len(all_original_players)} ({success_rate:.1f}%)")
        print(f"Total hero entries: {len(combined_hero_stats)}")
        print("Updated data saved to v2_data/player_stats_with_retries.csv and v2_data/player_hero_stats_with_retries.csv")
        
        return combined_stats, combined_hero_stats
    
    except FileNotFoundError:
        print("Error: Processed data files not found. Make sure the main processing completed and saved files.")
    except Exception as e:
        print(f"Error during retry process: {e}")


In [ ]:
# Rename as combined stats, retried feels like a subset, not full combined
retried_player_stats, retried_hero_stats = retry_failed_players(1,end_date=None,player_hero_stats=player_hero_stats, original_player_set=players, completed_player_stats=player_stats)

In [ ]:
retried_player_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_stats.csv", index=False)
retried_hero_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_hero_stats.csv", index=False)

In [ ]:
# another round of retries.

retried_player_stats, retried_hero_stats = retry_failed_players(1,end_date=None,player_hero_stats=retried_hero_stats, original_player_set=retried_player_stats, completed_player_stats=player_stats)

In [ ]:
retried_player_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_stats.csv", index=False)
retried_hero_stats.to_csv("v2_data\\8.12.data\\pred_data\\player_hero_stats.csv", index=False)

In [ ]:
# Checkpoint reset

# Read CSV files into DataFrames
matches = pd.read_csv(f"v2_data\\8.12.data\\pred_data\\pred_datamatches_2025-08-18.csv") # matches with outcome

player_hero_stats = pd.read_csv(f"v2_data\\8.12.data\\pred_data\\player_hero_stats.csv") # each players stats, broken down by hero

player_stats = pd.read_csv(f"v2_data\\8.12.data\\pred_data\\player_stats.csv") # aggregated player stats as totals (total kills, deaths, matches)

players = pd.read_csv(f"v2_data\\8.12.data\\pred_data\\pred_dataplayers_2025-08-18.csv") # player <> match <> hero data


In [ ]:
print(f"Matches csv: \n\n{matches.head()}")
print(f"Player Hero Stats csv: \n\n{player_hero_stats.head()}")
print(f"Player Stats csv: \n\n{player_stats.head()}")
print(f"Players csv: \n\n{players.head()}")

In [ ]:
# Merge player stats onto players (which is player+match data)
player_match_stats = pd.merge(players,player_stats, on="account_id",how="left")
print(f"Player Match Stats csv: \n\n{player_match_stats.head()}")

In [ ]:
player_match_stats.rename(columns={
    "kills": "pm_kills",
    "deaths": "pm_deaths",
    "assists": "pm_assists",
    "damage_per_min": "pm_damage_per_min",
    "denies": "pm_denies",
    "net_worth": "pm_net_worth",
    "win": "pm_win",
    'matches_played': 'p_total_matches_played',
    'total_kills': 'p_total_kills',
    'total_deaths': 'p_total_deaths',
    'total_assists': 'p_total_assists',
    'avg_kd': 'p_total_avg_kd',
    'win_rate': 'p_total_win_rate',
    'total_time_played': 'p_total_time_played'
},inplace=True)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_player_hero_stats(ph_stats_base: pd.DataFrame) -> pd.DataFrame:
    """
    Create player hero stats by aggregating the player_hero_stats DataFrame.
    Checks for potential divide by zero errors and sets result to 0 if denominator is zero.
    """
    ph_stats = pd.DataFrame()
    ph_stats = ph_stats_base.copy()

    # Avoid divide by zero for deaths
    ph_stats['ph_total_kd'] = np.where(ph_stats['deaths'] == 0, 0, ph_stats['kills'] / ph_stats['deaths'])

    ph_stats['h_total_kd'] = (ph_stats.groupby('hero_id')['ph_total_kd'].transform("mean"))
    # Avoid divide by zero for ph_total_kd
    ph_stats['ph_kd_ratio'] = np.where(ph_stats['ph_total_kd'] == 0, 0, ph_stats['h_total_kd']/ ph_stats['ph_total_kd'])

    ph_stats['h_avg_total_time_played'] = (ph_stats.groupby('hero_id')['time_played'].transform("mean"))
    # Avoid divide by zero for h_avg_total_time_played
    ph_stats['ph_time_played_ratio'] = np.where(ph_stats['h_avg_total_time_played'] == 0, 0, ph_stats['time_played']/ ph_stats['h_avg_total_time_played'])

    ph_stats['h_total_damage_per_min'] = (ph_stats.groupby('hero_id')['damage_per_min'].transform("mean"))
    # Avoid divide by zero for h_total_damage_per_min
    ph_stats['ph_damage_per_min_ratio'] = np.where(ph_stats['h_total_damage_per_min'] == 0, 0, ph_stats['damage_per_min']/ ph_stats['h_total_damage_per_min'])

    ph_stats['h_total_assists'] = (ph_stats.groupby('hero_id')['assists'].transform("mean"))
    # Avoid divide by zero for h_total_assists
    ph_stats['ph_assists_ratio'] = np.where(ph_stats['h_total_assists'] == 0, 0, ph_stats['assists']/ ph_stats['h_total_assists'])

    # Avoid divide by zero for matches_played
    ph_stats['ph_win_rate'] = np.where(ph_stats['matches_played'] == 0, 0, ph_stats['wins'] / ph_stats['matches_played'])
    ph_stats['h_total_win_rate'] = (ph_stats.groupby('hero_id')['ph_win_rate'].transform("mean"))
    # Avoid divide by zero for h_total_win_rate
    ph_stats['ph_win_rate_ratio'] = np.where(ph_stats['h_total_win_rate'] == 0, 0, ph_stats['ph_win_rate'] / ph_stats['h_total_win_rate'])

    ph_stats.rename(columns={
        "wins": "ph_wins",
        "kills": "ph_kills",
        "deaths": "ph_deaths",
        "assists": "ph_assists",
        "damage_per_min": "ph_damage_per_min",
        'time_played': 'ph_time_played'
    }, inplace=True)

    return ph_stats

In [ ]:
calc_player_hero_stats = create_player_hero_stats(player_hero_stats)
calc_player_hero_stats.head(10)

In [ ]:
calc_player_hero_stats.to_csv("quick_test.csv")

In [ ]:
# join player_match_stats on player_hero_stats on account_id AND hero_id
phm_stats = pd.merge(player_match_stats,calc_player_hero_stats,on=["hero_id", "account_id"], how="left")
phm_stats.head(11)

In [ ]:
phm_stats.to_csv("v2_data//8.12.data//pred_data//phm_stats.8.12_v2.csv")

In [ ]:
def create_std_team_stats(phm_stats_base: pd.DataFrame) -> pd.DataFrame:
    """
    Create team-level stats based on player&player_hero stats in min, max, mean, and std.
    """

    phm_stats = pd.DataFrame()
    phm_stats = phm_stats_base.copy()

    # set the team stats to be set to quantiles
    team_stats = [
        'p_total_time_played',
        'ph_wins','ph_kills','ph_deaths','ph_assists',
        'ph_damage_per_min','ph_time_played','ph_total_kd',
        'ph_kd_ratio', 'ph_time_played_ratio',
        'ph_damage_per_min_ratio','ph_assists_ratio','ph_win_rate_ratio'
    ]

    # check for missing columns
    missing_cols = [col for col in team_stats if col not in phm_stats.columns]
    if missing_cols:
        print(f"*CRITICAL* Missing columns in team stats: {missing_cols}")
        return pd.DataFrame()  # Return an empty DataFrame if missing columns are found

    # for each columm, set min, max, and quantiles
    agg_function = {
        col: ["min", "max",
                "mean",
                "std"]
        for col in team_stats
    }

    agg_function['pm_win'] = 'first'
    
    # group by columns, using agg_function as the aggregation function
    phm_stats = (phm_stats.groupby(
        ['match_id','team']).agg(
            agg_function))    
    
    # converts lambda tuples into col:val pairs
    def clean_columns(c_tuple):
        col, stat = c_tuple
        if stat == "min":
            return f"{col}_min"
        elif stat == "max":
            return f"{col}_max"
        elif stat == "mean":
            return f"{col}_mean"
        elif stat == "std":
            return f"{col}_std"
        elif callable(stat):
            return f"{col}_{stat.__name__}"
        else:
            return f"{col}_{stat}"

    phm_stats.columns = [clean_columns(col) for col in phm_stats.columns]

    return phm_stats.reset_index()

from datetime import datetime

now = datetime.now()
date = now.date()
hour = now.time()

team_stats = create_std_team_stats(phm_stats)
team_stats.to_csv(f"v2_data/team_stats_std_{date}.csv")

In [ ]:
def create_differential_training_data(team_stat_base:pd.DataFrame) -> pd.DataFrame:
    """
    Create training data by merging team stats with match outcomes.
    Each stat is a differential: Team0 - Team1
    """
    t_stats = team_stat_base.copy()
    t_stats = t_stats.pivot(index='match_id', columns='team')

    t_stats.columns = [f'{col[0]}_{col[1]}' for col in t_stats.columns]
    t_stats['team_0_win'] = t_stats['pm_win_first_Team0']
    t_stats.drop('pm_win_first_Team0', axis=1, inplace=True)
    t_stats.drop('pm_win_first_Team1', axis=1, inplace=True)
    
    # create differential columns
    for col in t_stats.columns:
        if col.endswith('_Team0'):
            base_col = col[:-6]  # remove '_Team0' to get the base column name
            team1_col = f'{base_col}_Team1'
            if team1_col in t_stats.columns:
                t_stats[f'{base_col}_diff'] = (t_stats[col] - t_stats[team1_col]).round(3)
                t_stats.drop(col, axis=1, inplace=True)
                t_stats.drop(team1_col, axis=1, inplace=True)

    t_stats = t_stats.reset_index()

    return t_stats

training_data = create_differential_training_data(team_stats)
training_data.to_csv(f"v2_data//8.12.data//pred_data///training_data_differentials_{date}.csv")
training_data.head(25)

In [ ]:
def check_data_issues(df):
    """Check for common missing/null/error values in the DataFrame and print summary. Also print rows with errors."""
    import numpy as np
    print("--- Data Issues Summary ---")
    # Check for NaN values
    nan_counts = df.isna().sum()
    if nan_counts.any():
        print("NaN values found:")
        print(nan_counts[nan_counts > 0])
    else:
        print("No NaN values found.")

    # Check for infinite values
    inf_counts = np.isinf(df.select_dtypes(include=[float, int])).sum()
    if inf_counts.any():
        print("Infinite values found:")
        print(inf_counts[inf_counts > 0])
    else:
        print("No infinite values found.")

    # Check for string 'inf', '-inf', 'nan', 'None', or empty string
    error_strings = ['inf', '-inf', 'nan', 'None', '']
    error_rows = set()
    for col in df.select_dtypes(include=[object]).columns:
        for err in error_strings:
            mask = (df[col] == err)
            count = mask.sum()
            if count > 0:
                print(f"Column '{col}' has {count} occurrences of '{err}'")
                error_rows.update(df[mask].index.tolist())

    # Collect all error rows (NaN, inf, error strings)
    nan_rows = set(df[df.isna().any(axis=1)].index.tolist())
    inf_rows = set(df[np.isinf(df.select_dtypes(include=[float, int])).any(axis=1)].index.tolist())
    all_error_rows = nan_rows.union(inf_rows).union(error_rows)

    print("--- End of Data Issues Summary ---")
    if all_error_rows:
        print(f"\nDetailed view of rows with data errors ({len(all_error_rows)} rows) saved to .csv:")
        df.loc[sorted(all_error_rows)].to_csv("data_errors.csv", index=False)
    else:
        print("No rows with data errors found.")

check_data_issues(training_data)